In [283]:
import matplotlib
matplotlib.use("Agg")
from neuralnilm.source import RealApplianceSource, MultiSource, SameLocation
import matplotlib.pyplot as plt
import numpy as np
import logging
import nilmtk
from datetime import timedelta

In [285]:
logger = logging.getLogger('test')
logger.setLevel(logging.DEBUG)

In [436]:
BUILDING_I = 2

MAX_TARGET_POWER = 3000
TARGET_APPLIANCE = 'microwave'
ON_POWER_THRESHOLD = 200
MIN_ON_DURATION = 12
MIN_OFF_DURATION = 60

SEQ_LENGTH = 128
N_SEQ_PER_BATCH = 8
FILENAME = '/data/mine/vadeec/merged/ukdale.h5'

WINDOWS_SELECTION = 'test'

if WINDOWS_SELECTION == 'short train':
    WINDOW_PER_BUILDING = {
        1: ("2013-04-12", "2013-04-18"),
        2: ("2013-05-22", "2013-06-01"),
        3: ("2013-02-27", "2013-03-10"),
        4: ("2013-03-09", "2013-04-01"),
        5: ("2014-06-29", "2014-07-10")
    }
elif WINDOWS_SELECTION == 'train':
    WINDOW_PER_BUILDING = {
        1: ("2013-03-17 19:12:43", "2014-12-15"),
        2: ("2013-05-22", "2013-10-03 06:16:00"),
        3: ("2013-02-27", "2013-04-01 06:15:05"),
        4: ("2013-03-09", "2013-09-24 06:15:14"),
        5: ("2014-06-29", "2014-09-01")
    }
elif WINDOWS_SELECTION == 'test':
    WINDOW_PER_BUILDING = {
        1: ("2014-12-15", "2014-12-22"),
        2: ("2013-10-03 06:16:00", None),
        3: ("2013-04-01 06:15:05", None),
        4: ("2013-09-24 06:15:14", None),
        5: ("2014-09-01", "2014-09-07")
    }

INPUT_STATS = {
    'mean': np.array([ 297.87216187], dtype=np.float32),
    'std': np.array([ 374.43884277], dtype=np.float32)
}

In [437]:
source = SameLocation(
    logger=logger,
    filename=FILENAME,
    target_appliance=TARGET_APPLIANCE,
    window_per_building=WINDOW_PER_BUILDING,
    seq_length=SEQ_LENGTH,
    train_buildings=[BUILDING_I],
    validation_buildings=[BUILDING_I],
    n_seq_per_batch=N_SEQ_PER_BATCH,
    load_mains=False,
    on_power_threshold=ON_POWER_THRESHOLD,
    min_on_duration=MIN_ON_DURATION,
    min_off_duration=MIN_OFF_DURATION
)

INFO:test:Setting window for building 2 to (start=2013-10-03 06:16:00, end=None)
INFO:test:Loaded 9 microwave activations from house 2.


In [438]:
seq_lengths = [(activation.index[-1] - activation.index[0])
               for activation in source.activations[BUILDING_I]]
max_duration = max(seq_lengths)
print("Max duration  :", max_duration)
print("Min input size:", max_duration.total_seconds() / 6)
print("Mean duration :", np.mean(seq_lengths))
print("Min duration  :", np.min(seq_lengths))
index_with_max_duration = np.argmax(seq_lengths)
print("Index with max duration :", index_with_max_duration)

Max duration  : 0 days 00:08:30
Min input size: 85.0
Mean duration : 0 days 00:02:58
Min duration  : 0 days 00:01:06
Index with max duration : 6


In [432]:
print("Max power  :", max([activation.max() for activation in source.activations[BUILDING_I]]))
print("Mean power :", np.mean([activation.mean() for activation in source.activations[BUILDING_I]]))

Max power  : 2859.0
Mean power : 1007.23532172


In [433]:
dataset = nilmtk.DataSet(FILENAME)
dataset.set_window()

elec = dataset.buildings[BUILDING_I].elec
meter = elec[TARGET_APPLIANCE]
meter

ElecMeter(instance=13, building=1, dataset='UK-DALE', appliances=[Appliance(type='microwave', instance=1)])

In [434]:
border = timedelta(minutes=10)

def plot_activation(activation_i):
    activation = source.activations[BUILDING_I][activation_i]
    timeframe = nilmtk.TimeFrame(activation.index[0]-border, activation.index[-1]+border)
    powers = {}
    elec = dataset.buildings[BUILDING_I].elec
    for meter in elec.submeters().meters:
        power = meter.power_series_all_data(sections=[timeframe])
        if power is not None and power.max() > 10:
            powers[meter.dominant_appliance().label(pretty=True)] = power
    
    fig, axes = plt.subplots(nrows=3, sharex=True)
    axes[0].set_title('Activation')
    axes[0].plot(activation.index, activation.values)

    axes[1].set_title('All active appliances')
    for meter_instance, power in powers.iteritems():
        axes[1].plot(power.index, power.values, label=meter_instance)
    axes[1].legend(fancybox=True, framealpha=0.5, prop={'size': 6})

    axes[2].set_title('Mains {}'.format(timeframe.start.date()))
    mains = elec.mains().power_series_all_data(sections=[timeframe], sample_period=6)
    if mains is not None:
        axes[2].plot(mains.index, mains.values)
    
    fig.tight_layout()
    fname = 'building{}_activation{}.png'.format(BUILDING_I, activation_i)
    plt.savefig(fname, dpi=300)
    plt.close(fig)

In [435]:
from sys import stdout
plot_activation(index_with_max_duration)
n = len(source.activations[BUILDING_I])
for i in range(n):
    print("\r", i, "/", n, end="")
    stdout.flush()
    plot_activation(i)

 52 / 90

Exception KeyboardInterrupt in <function remove at 0x7effac36e8c0> ignored


TypeError: sequence item 0: expected string, tuple found